# Mo' Numpy, Mo' Problems

This notebook is a quick overview of additional functionality in Numpy not explicitly covered in some of the other notebooks in this course.

In [ ]:
import numpy as np

# Random numbers

Numpy has a rich collection of (pseudo)random number generators. Here is an example; see the documentation for [numpy.random()](https://docs.scipy.org/doc/numpy/reference/routines.random.html) for more details.

In [ ]:
A = np.random.randint(-10, 10, size=(4, 3))
A

# Aggregations or reductions

Suppose you want to reduce the values of a Numpy array to a smaller number of values. Numpy provides a number of such functions that _aggregate_ values. Examples of aggregations include sums, min/max calculations, and averaging, among others.

In [ ]:
print(np.max(A), np.amax(A)) # np.max() and np.amax() are synonyms
print(np.min(A), np.amin(A)) # same
print(np.sum(A))
print(np.mean(A))
print(np.std(A))

The above examples aggregate over all values. But you can also aggregate along a dimension using the optional `axis` parameter.

In [ ]:
print("Max in each column:", np.amax(A, axis=0)) # i.e., aggregate along axis 0, the rows, producing column maximums
print("Max in each row:", np.amax(A, axis=1)) # i.e., aggregate along axis 1, the columns, producing row maximums

# Universal functions

Universal functions apply a given function _elementwise_ to one or more Numpy objects.

For instance, `np.abs(A)` takes the absolute value of each element.

In [ ]:
print(A, "\n==>\n", np.abs(A))

Some universal functions accept multiple, compatible arguments. Given two matrices, $A \equiv (a_{ij})$ and $B \equiv (b_{ij})$, the following example, computes a matrix $C$ such that $c_{ij} = \max(a_{ij}, b_{ij})$.

In [ ]:
B = np.random.randint(-10, 10, size=A.shape)
B

In [ ]:
C = np.maximum(A, B)
C

You can also build your own universal functions! For instance, suppose you want to compute, elementwise, $f(x) = e^{-x^2}$ and you have a scalar function that implements $f(x)$:

In [ ]:
def f(x):
    from math import exp
    return exp(-(x**2))

f(-2) # i.e., exp(-4) ~= 0.01831563888873418

This function accepts 1 input (`x`) and returns a single output. The following will create a new Numpy universal function.

In [ ]:
f_np = np.frompyfunc(f, 1, 1) # Creates a universal function from `f()`

print(A, "\n=>\n", f_np(A))

# Broadcasting

Sometimes we want to combine operations on Numpy arrays that have different shapes but are _compatible_.

In the following example, we want to add 3 elementwise to every value in `A`.

In [ ]:
print(A)
print()
print(A + 3)

Technically, `A` and `3` have different shapes: the former is a $4 \times 3$ matrix, while the latter is a scalar ($1 \times 1$). However, they are compatible because Numpy has a scheme to _extend_---or **broadcast**---the value 3 into an equivalent matrix object of the same shape, before combining them elementwise.

To see a more sophisticated example, suppose each row `A[i, :]` are the coordinates of a data point, and we want to compute the centroid (or "center-of-mass," if we imagine each point is a unit mass). That's the same as computing the mean coordinate for each column:

In [ ]:
A_row_means = np.mean(A, axis=0)
print(A, "=>", A_row_means)

Now, suppose you want to shift the points so that their mean is zero. Even though they don't have the same shape, Numpy will interpret `A - A_rowmeans` in a way that effectively carries out this operation. That is, it will extend or "replicate" `A_rowmeans` into rows of a matrix of the same shape as `A` and then perform elementwise subtraction.

In [ ]:
A_row_centered = A - A_row_means
A_row_centered

Suppose you instead want to mean-center the _columns_ instead of the rows. You could start by computing column means:

In [ ]:
A_col_means = np.mean(A, axis=1)
print(A, "=>", A_col_means)

But the same operation will fail!

In [ ]:
A - A_col_means # Fails, throwing a `ValueError`

The error reports that these shapes are not compatible. So how can you fix it?

**The broadcasting rule.** One way is to learn Numpy's convention for **[broadcasting](https://docs.scipy.org/doc/numpy/reference/ufuncs.html#broadcasting)**. Numpy starts by looking at the shapes of the objects:

In [ ]:
print(A.shape, A_row_means.shape)

These are compatible if, starting from _right_ to _left_, the dimensions match **or** one of the dimensions is 1. This convention of moving from right to left is referred to as matching the _trailing dimensions_. In this example, the rightmost dimensions of each object are both 3, so they match. Since `A_row_means` has no more dimensions, it can be replicated to match the remaining dimensions of `A`.

By contrast, consider the shapes of `A` and `A_col_means`:

In [ ]:
print(A.shape, A_col_means.shape)

In this case, per the broadcasting rule, the trailing dimensions of 3 and 4 do not match. Therefore, the broadcast rule fails. One way to get the desired behavior is to modify `A_col_means` to have a unit trailing dimension. In this case, you can use Numpy's [`reshape()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html) to convert `A_col_means` into a shape that has an explicit trailing dimension of size 1.

In [ ]:
A_col_means2 = np.reshape(A_col_means, (len(A_col_means), 1))
print(A_col_means2, "=>", A_col_means2.shape)

Now the trailing dimension equals 1, so it can be matched against the trailing dimension of `A`. The next dimension is the same between the two objects, so Numpy knows it can replicate accordingly.

In [ ]:
print(A, "\n-", A_col_means2)
print("=>\n", A - A_col_means2)

**Fin!** That marks the end of this notebook. If you want to learn more, check out the second edition of [Python for Data Analysis](http://shop.oreilly.com/product/0636920050896.do) (released in October 2017).